In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.utils import get_file
import numpy as np
import tensorflow as tf
import os

# 데이터 로드 및 전처리
path = get_file('shakespeare.txt', 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt')
text = open(path, 'rb').read().decode(encoding='utf-8')

# 고유 문자 집합 생성
chars = sorted(set(text))
char_to_index = dict((c, i) for i, c in enumerate(chars))
index_to_char = dict((i, c) for i, c in enumerate(chars))

# 텍스트를 숫자로 변환
max_length = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - max_length, step):
    sentences.append(text[i: i + max_length])
    next_chars.append(text[i + max_length])
x = np.zeros((len(sentences), max_length, len(chars)), dtype=np.bool_)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool_)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_to_index[char]] = 1
    y[i, char_to_index[next_chars[i]]] = 1

# 모델 구축
model = Sequential()
model.add(LSTM(128, input_shape=(max_length, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

# 모델 학습
model.fit(x, y, batch_size=128, epochs=10)

# 텍스트 생성 함수
def generate_text(model, length, diversity):
    start_index = np.random.randint(0, len(text) - max_length - 1)
    generated = ''
    sentence = text[start_index: start_index + max_length]
    generated += sentence
    for i in range(length):
        x_pred = np.zeros((1, max_length, len(chars)), dtype=np.bool_)
        for t, char in enumerate(sentence):
            x_pred[0, t, char_to_index[char]] = 1.
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = np.random.choice(len(chars), p=preds)
        next_char = index_to_char[next_index]
        generated += next_char
        sentence = sentence[1:] + next_char
    return generated


# 생성된 텍스트 출력
print(generate_text(model, length=400, diversity=1.0))

Epoch 1/10
2905/2905 [==============================] - 17s 5ms/step - loss: 2.3963
Epoch 2/10
2905/2905 [==============================] - 14s 5ms/step - loss: 1.9927
Epoch 3/10
2905/2905 [==============================] - 14s 5ms/step - loss: 1.8441
Epoch 4/10
2905/2905 [==============================] - 14s 5ms/step - loss: 1.7538
Epoch 5/10
2905/2905 [==============================] - 15s 5ms/step - loss: 1.6910
Epoch 6/10
2905/2905 [==============================] - 14s 5ms/step - loss: 1.6450
Epoch 7/10
2905/2905 [==============================] - 14s 5ms/step - loss: 1.6087
Epoch 8/10
2905/2905 [==============================] - 14s 5ms/step - loss: 1.5786
Epoch 9/10
2905/2905 [==============================] - 14s 5ms/step - loss: 1.5532
Epoch 10/10
2905/2905 [==============================] - 15s 5ms/step - loss: 1.5317
 him be prepared;
For that's the utmost to de a careling,
To wistcail'd mind orf, I hine love tell mormenck,
And sand the liftors 'tming diething your higr fra

In [2]:
# 모델 저장
model.save('shakespeare_text_gen_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
